# Batch Normalization

**What is Batch Normalization**

1. Batch-Normalization (BN) is an algorithmic method which makes 
the training of Deep Neural Networks (DNN) **faster** and more 
**stable**.

2. It consists of normalizing activation vectors from hidden 
layers using the **mean and variance** of the current batch. 

3. This normalization step is applied right before (or right after) the 
nonlinear function.





**Why use BatchNorm** 

1. In case of unnormalized data , contour plot is like oval and in that case if you keep learning rate high then it overshoots , so we keep lr low which results in slow training.
(just check dropout note first slide for how it looks ).

2. In case of normalized data , contour plot is like circle , in which you can keep lr high , that makes training faster.

3. Due to internal covariate shift , input distribution for furthers layers keep changing , 
 due to thi slater layers constantly need to re-adapt and hence leads to instability and lower training also.





**Concept of Internal Covariate Shift**

1. First understand Covariate Shift , it happens when 
    
    1. Input distribution changes

    2. But relationship between input → output stays the same

2. For example : 
   
    1. Train only on red roses → model learns “rose = red + flower features”
    
    2. Later see blue/yellow roses → still roses, but input distribution changed.
    
    3. Model struggles because it never saw these distributions before . So the model has to re-adapt to new types of roses, even though the labeling rule didn’t change.

3. Internal Covariate shift is defined as change in distribution of network activations due to change in network parameters during training.

4. Internal Covariate Shift (ICS) : Same concept, but happening inside the neural network.
   Example : Inside layer 5 of a network , Input distribution keeps changing every batch Because layers 1–4 weights are updating continuously. So deeper layers are constantly receiving different feature distributions, causing : Unstable training ,Slower convergence.


**How BatchNorm Works ?** 

check slides fo example.

1. z11 -> z11(N) -> z11(BN) -> g(z11(BN)) -> a11 (check Batch Normalization slides).

2. z11 = w1 * (cgpa) + w2 * (iq) + b . Shape: (4 × 2) For each neuron, we normalize across batch (down each column) [4 is batch size] .[mu and sigma i.e mean and std devn  is not learnable]  [mean = 0 , std devn = 1 , (std devn)^2 = variance = 1]

3. z11(BN) = gamma * (z11(N)) + beta * (z11(N)) , then further g(z11(NB)) is activation , 
where gamman and beta are learnable paramaters.

4. While normalizing z11 , we add epsilon in denominator so that in case sigma get 0 , it handles.


**Batch Norm during test**

1. During testing we are providing the data in batches , but in test we provide a single row , then in that case how is z11 normalized ? (as we required mean and std devn and for single row how they can be calculated).

2. For that , we calculated mean and variance using exponential weighted meaning average.For each neuron seperately.

3. check for expression on internet its like [val_store = alpha * val_store + (1-alpha)*current_caluculated] and for std devn conisder square. [where alpha is momentum , a hyper parameter]

**Advantages**

1. Make training stable.

2. Make training faster , as you can choose high lr.

3. Act as regularizer in some sense . (not too much extent like dropout).[like value of mean and std devn depends on batch itself (if batch changes) , then lead to changes in activation and that can introduce a little randomness or some noise which leads to little decrase in overfitting].

4. Not to worry about Weight initialization , as its impact reduce because now normalization happens so it converges (check slides).  [cost function stretched without using it and now it takes time to reach optimal solution , but if you used it , then it is uniform , then you can reach optimal solution in a better way.]

# DropOuts

**What is DropOut and how it works**

**How it works in testing** 

**DrawBacks**

# L2 Regularization

**What is Regularization**



# Code